In [21]:
import networkx as nx
import scripts.read_graph as rg
import folium
import random

In [2]:
def read(path):

    MG = nx.read_gexf(path)

    for node in MG.nodes(data=True):
        for key, value in MG.edge[node[0]].items():
            for k, v in value.items():
                if type(v['attr']) == str:
                    MG.edge[node[0]][key][k]['attr'] = ast.literal_eval(v['attr'])

    for node in MG.nodes(data=True):
        if isinstance(MG.node[node[0]]['location'], str):
            MG.node[node[0]]['location'] = ast.literal_eval(MG.node[node[0]]['location'])
            if not isinstance(MG.node[node[0]]['location'], dict):
                print('NAO FUNCIONA')
                break
        if len(node[1]) == 3:
            if type(MG.node[node[0]]['viagens']) == str:
                MG.node[node[0]]['viagens'] = ast.literal_eval(MG.node[node[0]]['viagens'])

    return MG

In [3]:
MG = rg.read('graphs/sp_final.gexf')

Não deixar o fera ficar entrando e saindo de busao

In [4]:
def getpath(nested_dict, value, prepath=()):
    for k, v in nested_dict.items():
        path = prepath + (k,)
        if v == value: # found value
            return path
        elif hasattr(v, 'items'): # v is a dict
            p = getpath(v, value, path) # recursive call
            if p is not None:
                return p

In [85]:
class Walker():
    
    def __init__(self, graph, stop_id, travel_time, current_time):
        self.MG = graph 
        self.stop_id = stop_id 
        self.total_time_travel = travel_time * 60 # minutos para segundos 
        self.traveled_time = 0 
        self.visited_set = [] 
        self.tentative_value = {} 
        self.current_time = current_time * 60 * 60 # hora para segundo 
        
        self.calculate_location = lambda x: (MG.node[x]['location']['lat'], MG.node[x]['location']['long'])# tuple
        self.current_location = self.calculate_location(stop_id)
    
    def zera_tempo(self):
        self.traveled_time = 0
        
    def muda_id(self,stop_id):
        self.stop_id = stop_id
    
    def build_map(self):
        
        stops = [self.MG.node[i] for i in self.visited_set]
        COORD = (stops[0]['location']['lat'],stops[0]['location']['long']) # sp coordinates
        mapa = folium.Map(location=COORD, zoom_start=12)

        for i, stop in enumerate(stops[1:]):
            
            if i == 0:
                color = 'black'
                fill_color = 'red'
                radius=5
            elif i < len(self.visited_set):
                fill_color = 'black'
                color = 'black'
                radius=1
                

            folium.RegularPolygonMarker(
                location = [stop['location']['lat'],stop['location']['long']],
                color=color,
                fill_color=fill_color,
                number_of_sides=100,
                radius=radius,
                ).add_to(mapa)

        return(mapa)
    
    
    def isochrone(self):
        i=0
        while 1:
            #print('\n{}: '.format(i))
            self.find_neighbours()

            minimum, min_key = -1, 0
            for key, values in self.tentative_value.items():
                if key not in self.visited_set:
                    if minimum == -1:
                        minimum = values['travel_time']
                        min_key = key
                    elif values['travel_time'] < minimum:
                        minimum = values['travel_time']
                        min_key = key
            
            
            self.stop_id = min_key
            self.traveled_time = self.traveled_time + minimum
            self.current_time = self.current_time + minimum
            
            #print('\n--------\nBusao:', self.tentative_value[min_key]['linha'])
            #print('Tentative_Values', self.tentative_value)
            #print('Visited set', self.visited_set)
            #print('Traveled Time: ',self.traveled_time)
            if self.traveled_time > self.total_time_travel:
            #    print('FINAL:', self.stop_id)
                break
            i=i+1
    def random_isochrone(self):
        
        while 1:
        
            linha_atual = 0
                
            vizinhos = list(self.MG.node[self.stop_id]['viagens'].keys())
            
            if len(vizinhos)==0:
                print("Chegou em um ponto onde nao tem linhas para pegar")
                break
            
            vizinho_aleatorio = random.choice(vizinhos)
            
            #print("Opcoes: ",vizinhos)
            
            valores_vizinho_aleatorio = (self.MG.node[self.stop_id]['viagens'][vizinho_aleatorio])
            
            travel_time =  (min(valores_vizinho_aleatorio,
                                  key=lambda x: abs(x-self.current_time) if x-self.current_time<=0 else False) -
                                self.current_time)
            
            # have to be greater than current_time
            if travel_time < 0:
                continue
            
            # get time to next stop  
            output = getpath(self.MG.edge[self.stop_id], vizinho_aleatorio)
            if output == None:
                continue
            
            
            output = getpath(self.MG.edge[self.stop_id], vizinho_aleatorio)
            stop_id_next, trip_key, *rest = output
                
            trip_time = self.MG.edge[self.stop_id][stop_id_next][trip_key]['attr']['tempo']
            travel_time = travel_time + trip_time
                
            #print("Trip Time: ", trip_time)
            #print("Total Time: ", travel_time)
            #print('Stop Next', stop_id_next, "\n")
                
            self.visited_set.append(self.stop_id)
            
            self.stop_id = stop_id_next
            self.traveled_time = self.traveled_time + trip_time
            self.current_time = self.current_time + trip_time
            
            #print("Todos locais passados: ", self.visited_set)
            #print("Atual: ",self.stop_id)
            #print("Tempo Total: ",self.traveled_time)
            
            if self.traveled_time > self.total_time_travel:
                print('FINAL:', self.stop_id)
                break    
        
            
    def find_neighbours(self):
        """Given a stop, find neighbours and calculate travel time. 
        Two types of neighbours: connectionable and walkable
        """
        #print('StopId: ', self.stop_id)
        #pprint(self.current_time)
        #get connection neighbours
        linha_atual = 0
        if len(self.visited_set) > 0:
            linha_atual = self.tentative_value[self.stop_id]['linha']
            # print('Linha Atual:', linha_atual)
            
        for conn_neigh_key, conn_neigh_value in self.MG.node[self.stop_id]['viagens'].items():
            
            #print('TripId', conn_neigh_key)
            # get next travel time
            # print("------\nLine Name: ", conn_neigh_key, "\n")
            #print(conn_neigh_value)
            travel_time =  (min(conn_neigh_value,
                              key=lambda x: abs(x-self.current_time) if x-self.current_time<=0 else False) -
                            self.current_time)
            """
            if conn_neigh_key == linha_atual:
                pass
                print("----Current Line:", travel_time)
            else:
                pass
                print("Other Line:", travel_time)
            """           
            # have to be greater than current_time
            if travel_time < 0:
                continue
            
            # get time to next stop  
            output = getpath(self.MG.edge[self.stop_id], conn_neigh_key)
            if output == None:
                continue
            stop_id_next, trip_key, *rest = output
            
            if stop_id_next in self.visited_set:
                continue
            trip_time = self.MG.edge[self.stop_id][stop_id_next][trip_key]['attr']['tempo']
            travel_time = travel_time + trip_time
            """
            print("Trip Time: ", trip_time)
            print("Total Time: ", travel_time)
            print('Stop Next', stop_id_next, "\n")
            """
            # check travel time to next stop and replace if smaller
            if stop_id_next in self.tentative_value:
                if travel_time < self.tentative_value[stop_id_next]['travel_time']:
                    self.tentative_value[stop_id_next] = {'travel_time': travel_time,
                                                          'trip_key': trip_key,
                                                          'previous_stop': self.stop_id,
                                                        'linha': conn_neigh_key}
            else:
                self.tentative_value[stop_id_next] = {'travel_time': travel_time,
                                                      'trip_key': trip_key,
                                                      'previous_stop': self.stop_id,
                                                     'linha': conn_neigh_key}
   
            #print(self.tentative_value)
    
        self.visited_set.append(self.stop_id)
        
        # get walkable neighbours
        

In [93]:
node_id = '706304'
walker = Walker(MG, node_id, travel_time=60, current_time=7)

for num_rand_walks in range(0,10):

    walker.random_isochrone()
    walker.zera_tempo()
    walker.muda_id('706304')



walker.build_map()

FINAL: 960010719
FINAL: 440015107
FINAL: 900014673
FINAL: 6714583
FINAL: 340015736
FINAL: 660017173
FINAL: 5013154
FINAL: 480014721
FINAL: 660017173
FINAL: 340015374


In [53]:
node_id = '706304'
walker = Walker(MG, node_id, travel_time=60, current_time=7)
walker.isochrone()
walker.build_map()